In [47]:
import pandas as pd 
import numpy as np

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# Method 1: Using Pandas (Recommended for data cleaning)
# Read the Excel file
df = pd.read_excel("E:\\Lenguard-Second project\\rent_roll_files\\Abbie Lakes - Rent Roll - 5.15.24 (Lender Pulled) ajr.xlsx")

# Remove rows where all values are NaN
df_clean = df.dropna(how='all')

# Remove columns where all values are NaN
df_clean = df_clean.dropna(axis=1, how='all')

df_clean

property_info = df_clean.iloc[0:4]

# Get the first column with property info headers
property_info_headers = property_info['Abbie Lakes'].values
print("Property Information :")
for header in property_info_headers:
    print(header)


rent_roll_data = df_clean.iloc[4:]

# Set the 4th row (index 4) as the header for rent roll data
rent_roll_data.columns = rent_roll_data.iloc[0]

# Remove the header row from the data
rent_roll_data = rent_roll_data.iloc[1:].reset_index(drop=True)

# Find rows containing 'total' (case insensitive)
total_rows = rent_roll_data[rent_roll_data.apply(lambda x: x.astype(str).str.contains('total', case=False)).any(axis=1)]

# # Remove total rows from main data
rent_roll_data = rent_roll_data[~rent_roll_data.index.isin(total_rows.index)]

total_rows.iloc[:1]

# Get the first row of total_rows and drop NaN values
total_row_clean = total_rows.iloc[0].dropna()

# Create a new DataFrame with just the non-NaN values
total_summary = pd.DataFrame([total_row_clean]).reset_index(drop=True)

# Display the cleaned total row
print("\nCleaned Total Row Summary:")
print(total_summary)



Property Information :
RENT ROLL DETAIL MODIFIED
5/16/2024 3:11:55 AM
As of Date: 5/15/2024
Parameters: Property - ALL; Unit Designation - ALL; Subjournal - ALL; Sort By - Unit; Display - Market + Addl.

Cleaned Total Row Summary:
4 Resh ID Market + Addl. Dep On Hand    Balance Total Charges  \
0   TOTAL     285,137.00   60,189.83  -1,774.23    261,921.20   

4 RENT                           PEST                            \
0                     251,448.00                         630.00   

4 VALET TRASH                    TRASH                           \
0                       6,374.00                       1,682.00   

4 PETRENT                        LOP15                           \
0                       1,825.00                         795.00   

4 EMPLCRED                       OHIOCHOICE                      \
0                        -657.80                           0.00   

4 WAIVED ADMIN FEE               WAIVED APP FEE                  \
0                        -150.0

In [48]:
rent_roll_data

4,Resh ID,Lease ID,Unit,Floor Plan,Unit Designation,SQFT,Unit/Lease Status,Name,Phone Number,Email,Move-In,Notice For Date,Move-Out,Lease Start,Lease End,Market + Addl.,Dep On Hand,Balance,Total Charges,RENT,PEST,VALET TRASH,TRASH,PETRENT,LOP15,EMPLCRED,OHIOCHOICE,WAIVED ADMIN FEE,WAIVED APP FEE,SETUPFEE
0,101887,102845,3680,B2R,NaN,1032,Occupied,"Wray, Joe",614-815-9588,joepwrayjr@gmail.com,02/29/2024,NaN,NaN,02/29/2024,02/20/2025,1414.00,300.00,0.00,1338.00,1296.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,91810,92712,3682,B2R,NaN,1032,Occupied,"Wright, Patty",(614) 634-8523,pwright3611@gmail.com,07/17/2023,NaN,NaN,07/17/2023,09/16/2024,1459.00,300.00,0.00,1372.00,1331.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,101838,102758,3684,B2R,NaN,1032,Occupied,"Barry, Abdoulaye",646-403-7710,bademba.barrie89@live.com,08/02/2023,NaN,NaN,08/02/2023,10/30/2024,1364.00,300.00,0.00,1393.00,1337.00,3.00,30.00,8.00,0.00,15.00,0.00,0.00,0.00,0.00,0.00
3,101936,102913,3686,B2R,NaN,1032,Occupied,"Ivory, Rashida",347-255-8785,ivoryrashida@gmail.com,03/11/2024,NaN,NaN,03/11/2024,03/10/2025,1409.00,300.00,0.00,1380.00,1303.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
4,101939,102918,3688,B1R,NaN,990,Occupied,"Cooper, Tyreke",(614) 556-7982,tyreke68@gmail.com,03/20/2024,NaN,NaN,03/20/2024,04/17/2025,1343.00,300.00,0.00,1339.00,1297.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,101855,102886,3690,B1R,NaN,990,Occupied,"Lawrence, Lisa",614-204-1704,gmalisa17@gmail.com,09/07/2023,NaN,NaN,04/05/2024,11/11/2024,1363.00,300.00,-1508.57,1364.00,1287.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
6,91679,102829,3692,A1R,NaN,806,Occupied-NTVL,"Bouillon, Douglas",419-937-5272,dbouillon07@yahoo.com,10/04/2022,06/05/2024,NaN,01/06/2024,06/05/2024,1189.00,300.00,-224.49,1272.00,1181.00,3.00,30.00,8.00,35.00,15.00,0.00,0.00,0.00,0.00,0.00
7,101948,102936,3694,A1R,NaN,806,Occupied,"Sanker, Matthew",380-250-8029,ggggoal1957@gmail.com,04/06/2024,NaN,NaN,04/06/2024,05/07/2025,1169.00,300.00,0.00,1169.00,1127.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,30,92577,3696,B1R,NaN,990,Occupied,"Smith, William",(614) 216-1689,willwsmith1966@gmail.com,08/01/2011,NaN,NaN,04/09/2023,06/08/2024,1293.00,199.00,0.00,1220.00,1179.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,50784,92698,3698,B1R,NaN,990,Occupied,"Cruz, Marlene",(614) 984-4116,cruzmarlene044@gmail.com,02/03/2018,NaN,NaN,07/24/2023,07/16/2024,1313.00,100.00,0.00,1264.00,1223.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Rent Roll Processor
#
# Functions:
# - load_excel: Load file
# - parse_data: Convert to df
# - find_headers: Get headers
# - get_metadata: Property info
# - process_data: Clean data
#
# Purpose: Extract property & rent roll data


In [49]:
! pip install "pydantic>=2.0.0" "openai>=1.0.0"

In [50]:
import pandas as pd

def load_excel_data(file_path: str) -> pd.ExcelFile:
    """Load Excel file from given path"""
    return pd.ExcelFile(file_path)

def parse_excel_data(excel_file: pd.ExcelFile) -> pd.DataFrame:
    """Parse Excel file into DataFrame"""
    return excel_file.parse()

def find_header_row(data: pd.DataFrame, headers: list) -> int:
    """Find the row containing any of the expected headers"""
    header_row_idx = None
    for i, row in data.iterrows():
        if any(col in headers for col in row.astype(str)):  # Check if any column matches expected headers
            header_row_idx = i
            break  # Stop after finding the first match

    if header_row_idx is None:
        raise ValueError("No matching header row found in the Excel file.")
    
    print(f"Header row found at index: {header_row_idx}")
    return header_row_idx
def extract_metadata(data: pd.DataFrame, header_index: int) -> dict:
    """Extract and process metadata before header row"""
    metadata = data.iloc[:header_index].copy()
    metadata = metadata.dropna(how='all').reset_index(drop=True)
    
    metadata_dict = {}
    for idx, row in metadata.iterrows():
        row_clean = row.dropna()
        if not row_clean.empty:
            metadata_dict[f"row_{idx}"] = row_clean.to_dict()
    return metadata_dict

def process_rent_roll_data(data: pd.DataFrame, header_index: int) -> pd.DataFrame:
    """Process rent roll data after header row"""
    data = data.iloc[header_index:].reset_index(drop=True)
    data.columns = data.iloc[0]
    rent_roll_data = data.iloc[1:].reset_index(drop=True)
    return rent_roll_data.dropna(how='all').reset_index(drop=True)
def main():
    # Define expected headers - any one of these should be present
    headers = [
        'Resh ID', 'Lease ID', 'Unit', 'Floor Plan', 'Unit Designation', 'SQFT',
        'Unit/Lease Status', 'Name', 'Phone Number', 'Email', 'Move-In', 
        'Notice For Date', 'Move-Out', 'Lease Start', 'Lease End', 'Market + Addl.',
        'Dep On Hand', 'Balance', 'Total Charges', 'RENT', 'PEST', 'VALET TRASH',
        'TRASH', 'PETRENT', 'LOP15', 'EMPLCRED', 'OHIOCHOICE', 'WAIVED ADMIN FEE',
        'WAIVED APP FEE', 'SETUPFEE'
    ]
    
    # Pipeline execution
    file_path = input("Please enter the path to your rent roll Excel file: ")
    excel_file = load_excel_data(file_path)
    raw_data = parse_excel_data(excel_file)
    header_index = find_header_row(raw_data, headers)
    
    if header_index is not None:
        metadata = extract_metadata(raw_data, header_index)
        rent_roll_data = process_rent_roll_data(raw_data, header_index)
        
        print("Metadata:")
        print(metadata.values())
        
        return rent_roll_data
    else:
        raise ValueError("Could not find any of the expected headers in data")

# Execute pipeline
rent_roll_data = main()
rent_roll_data


Header row found at index: 4
Metadata:
dict_values([{'Abbie Lakes': 'RENT ROLL DETAIL MODIFIED'}, {'Abbie Lakes': '5/16/2024 3:11:55 AM'}, {'Abbie Lakes': 'As of Date: 5/15/2024'}, {'Abbie Lakes': 'Parameters: Property - ALL; Unit Designation - ALL; Subjournal - ALL; Sort By - Unit; Display - Market + Addl.'}])


,Resh ID,Lease ID,Unit,Floor Plan,Unit Designation,SQFT,Unit/Lease Status,Name,Phone Number,Email,Move-In,Notice For Date,Move-Out,Lease Start,Lease End,Market + Addl.,Dep On Hand,Balance,Total Charges,RENT,PEST,VALET TRASH,TRASH,PETRENT,LOP15,EMPLCRED,OHIOCHOICE,WAIVED ADMIN FEE,WAIVED APP FEE,SETUPFEE
0,101887,102845,3680,B2R,NaN,1032,Occupied,"Wray, Joe",614-815-9588,joepwrayjr@gmail.com,02/29/2024,NaN,NaN,02/29/2024,02/20/2025,1414.00,300.00,0.00,1338.00,1296.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,91810,92712,3682,B2R,NaN,1032,Occupied,"Wright, Patty",(614) 634-8523,pwright3611@gmail.com,07/17/2023,NaN,NaN,07/17/2023,09/16/2024,1459.00,300.00,0.00,1372.00,1331.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,101838,102758,3684,B2R,NaN,1032,Occupied,"Barry, Abdoulaye",646-403-7710,bademba.barrie89@live.com,08/02/2023,NaN,NaN,08/02/2023,10/30/2024,1364.00,300.00,0.00,1393.00,1337.00,3.00,30.00,8.00,0.00,15.00,0.00,0.00,0.00,0.00,0.00
3,101936,102913,3686,B2R,NaN,1032,Occupied,"Ivory, Rashida",347-255-8785,ivoryrashida@gmail.com,03/11/2024,NaN,NaN,03/11/2024,03/10/2025,1409.00,300.00,0.00,1380.00,1303.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
4,101939,102918,3688,B1R,NaN,990,Occupied,"Cooper, Tyreke",(614) 556-7982,tyreke68@gmail.com,03/20/2024,NaN,NaN,03/20/2024,04/17/2025,1343.00,300.00,0.00,1339.00,1297.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,101855,102886,3690,B1R,NaN,990,Occupied,"Lawrence, Lisa",614-204-1704,gmalisa17@gmail.com,09/07/2023,NaN,NaN,04/05/2024,11/11/2024,1363.00,300.00,-1508.57,1364.00,1287.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
6,91679,102829,3692,A1R,NaN,806,Occupied-NTVL,"Bouillon, Douglas",419-937-5272,dbouillon07@yahoo.com,10/04/2022,06/05/2024,NaN,01/06/2024,06/05/2024,1189.00,300.00,-224.49,1272.00,1181.00,3.00,30.00,8.00,35.00,15.00,0.00,0.00,0.00,0.00,0.00
7,101948,102936,3694,A1R,NaN,806,Occupied,"Sanker, Matthew",380-250-8029,ggggoal1957@gmail.com,04/06/2024,NaN,NaN,04/06/2024,05/07/2025,1169.00,300.00,0.00,1169.00,1127.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,30,92577,3696,B1R,NaN,990,Occupied,"Smith, William",(614) 216-1689,willwsmith1966@gmail.com,08/01/2011,NaN,NaN,04/09/2023,06/08/2024,1293.00,199.00,0.00,1220.00,1179.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,50784,92698,3698,B1R,NaN,990,Occupied,"Cruz, Marlene",(614) 984-4116,cruzmarlene044@gmail.com,02/03/2018,NaN,NaN,07/24/2023,07/16/2024,1313.00,100.00,0.00,1264.00,1223.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [51]:
! pip install python-calamine 

In [52]:
file_path = input("Please enter the path to your rent roll Excel file: ")
abbie = pd.ExcelFile(file_path)

df = abbie.parse()

df.head()


,Abbie Lakes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,RENT ROLL DETAIL MODIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5/16/2024 3:11:55 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,As of Date: 5/15/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Parameters: Property - ALL; Unit Designation - ALL; Subjournal - ALL; Sort By - Unit; Display - Market + Addl.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Resh ID,Lease ID,Unit,Floor Plan,Unit Designation,SQFT,Unit/Lease Status,Name,Phone Number,Email,Move-In,Notice For Date,Move-Out,Lease Start,Lease End,Market + Addl.,Dep On Hand,Balance,Total Charges,RENT,PEST,VALET TRASH,TRASH,PETRENT,LOP15,EMPLCRED,OHIOCHOICE,WAIVED ADMIN FEE,WAIVED APP FEE,SETUPFEE


In [53]:
headers = [
    'Resh ID', 'Lease ID', 'Unit', 'Floor Plan', 'Unit Designation', 'SQFT',
    'Unit/Lease Status', 'Name', 'Phone Number', 'Email', 'Move-In',
    'Notice For Date', 'Move-Out', 'Lease Start', 'Lease End', 'Market + Addl.',
    'Dep On Hand', 'Balance', 'Total Charges'
]


In [54]:

 # Iterate through rows to find the header row
header_row_idx = None
for i, row in df.iterrows():
        if any(col in headers for col in row.astype(str)):  # Check if any column matches expected headers
            header_row_idx = i
            break  # Stop after finding the first match

if header_row_idx is None:
        raise ValueError("No matching header row found in the Excel file.")
# Print header row index and contents
print(f"Header row found at index: {header_row_idx}")
# print("\nHeader row contents:")
# print(df.iloc[header_row_idx].to_string())

# Split DataFrame from the header row onwards
df = df.iloc[header_row_idx:]  # Keep rows from header row onwards
df.columns = df.iloc[0]  # Set the first row as column names
rent_roll_data = df[1:].reset_index(drop=True)  # Drop the header row from data and reset index

rent_roll_data  # Return cleaned DataFrame


Header row found at index: 4


4,Resh ID,Lease ID,Unit,Floor Plan,Unit Designation,SQFT,Unit/Lease Status,Name,Phone Number,Email,Move-In,Notice For Date,Move-Out,Lease Start,Lease End,Market + Addl.,Dep On Hand,Balance,Total Charges,RENT,PEST,VALET TRASH,TRASH,PETRENT,LOP15,EMPLCRED,OHIOCHOICE,WAIVED ADMIN FEE,WAIVED APP FEE,SETUPFEE
0,101887,102845,3680,B2R,NaN,1032,Occupied,"Wray, Joe",614-815-9588,joepwrayjr@gmail.com,02/29/2024,NaN,NaN,02/29/2024,02/20/2025,1414.00,300.00,0.00,1338.00,1296.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,91810,92712,3682,B2R,NaN,1032,Occupied,"Wright, Patty",(614) 634-8523,pwright3611@gmail.com,07/17/2023,NaN,NaN,07/17/2023,09/16/2024,1459.00,300.00,0.00,1372.00,1331.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,101838,102758,3684,B2R,NaN,1032,Occupied,"Barry, Abdoulaye",646-403-7710,bademba.barrie89@live.com,08/02/2023,NaN,NaN,08/02/2023,10/30/2024,1364.00,300.00,0.00,1393.00,1337.00,3.00,30.00,8.00,0.00,15.00,0.00,0.00,0.00,0.00,0.00
3,101936,102913,3686,B2R,NaN,1032,Occupied,"Ivory, Rashida",347-255-8785,ivoryrashida@gmail.com,03/11/2024,NaN,NaN,03/11/2024,03/10/2025,1409.00,300.00,0.00,1380.00,1303.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
4,101939,102918,3688,B1R,NaN,990,Occupied,"Cooper, Tyreke",(614) 556-7982,tyreke68@gmail.com,03/20/2024,NaN,NaN,03/20/2024,04/17/2025,1343.00,300.00,0.00,1339.00,1297.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,101855,102886,3690,B1R,NaN,990,Occupied,"Lawrence, Lisa",614-204-1704,gmalisa17@gmail.com,09/07/2023,NaN,NaN,04/05/2024,11/11/2024,1363.00,300.00,-1508.57,1364.00,1287.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
6,91679,102829,3692,A1R,NaN,806,Occupied-NTVL,"Bouillon, Douglas",419-937-5272,dbouillon07@yahoo.com,10/04/2022,06/05/2024,NaN,01/06/2024,06/05/2024,1189.00,300.00,-224.49,1272.00,1181.00,3.00,30.00,8.00,35.00,15.00,0.00,0.00,0.00,0.00,0.00
7,101948,102936,3694,A1R,NaN,806,Occupied,"Sanker, Matthew",380-250-8029,ggggoal1957@gmail.com,04/06/2024,NaN,NaN,04/06/2024,05/07/2025,1169.00,300.00,0.00,1169.00,1127.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,30,92577,3696,B1R,NaN,990,Occupied,"Smith, William",(614) 216-1689,willwsmith1966@gmail.com,08/01/2011,NaN,NaN,04/09/2023,06/08/2024,1293.00,199.00,0.00,1220.00,1179.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,50784,92698,3698,B1R,NaN,990,Occupied,"Cruz, Marlene",(614) 984-4116,cruzmarlene044@gmail.com,02/03/2018,NaN,NaN,07/24/2023,07/16/2024,1313.00,100.00,0.00,1264.00,1223.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


#### Using the openai for the metadata extraction 

In [55]:
# import pandas as pd
# import json
# import os
# import dotenv
# from openai import OpenAI
# import logging

# logger = logging.getLogger(__name__)


# def initialize_openai_client():
#     """Initialize OpenAI client with proper error handling and configuration."""
#     try:
#         # Load environment variables
#         dotenv.load_dotenv()
        
#         # Get API key with validation
#         api_key = os.getenv("OPENAI_API_KEY")
#         if not api_key:
#             raise ValueError("OpenAI API key not found in environment variables")
        
#         # Initialize client with configuration
#         client = OpenAI(
#             api_key=api_key,
#             timeout=60.0,  # Increase timeout for longer operations
#             max_retries=3  # Automatic retries for reliability
#         )
        
#         logger.info("OpenAI client initialized successfully")
#         return client
    
#     except Exception as e:
#         logger.error(f"Failed to initialize OpenAI client: {str(e)}")
#         raise
    
    
# def load_excel_data(file_path: str) -> pd.ExcelFile:
#     """Load Excel file from given path"""
#     return pd.ExcelFile(file_path)

# def parse_excel_data(excel_file: pd.ExcelFile) -> pd.DataFrame:
#     """Parse Excel file into DataFrame"""
#     return excel_file.parse()

# def find_header_row(data: pd.DataFrame, headers: list) -> int:
#     """Find the row containing any of the expected headers"""
#     for idx, row in data.iterrows():
#         row_str = ' '.join(str(val) for val in row.values)
#         # Check if ANY header is present instead of ALL
#         if any(header in row_str for header in headers):
#             # print(f"Headers found at row index: {idx}")
#             return idx
#     return None

# def extract_metadata(data: pd.DataFrame, header_index: int) -> dict:
#     """Extract and process metadata before header row"""
#     metadata = data.iloc[:header_index].copy()
#     metadata = metadata.dropna(how='all').reset_index(drop=True)
    
#     metadata_dict = {}
#     for idx, row in metadata.iterrows():
#         row_clean = row.dropna()
#         if not row_clean.empty:
#             metadata_dict[f"row_{idx}"] = row_clean.to_dict()
#     return metadata_dict

# def process_rent_roll_data(data: pd.DataFrame, header_index: int) -> pd.DataFrame:
#     """Process rent roll data after header row"""
#     data = data.iloc[header_index:].reset_index(drop=True)
#     data.columns = data.iloc[0]
#     rent_roll_data = data.iloc[1:].reset_index(drop=True)
#     return rent_roll_data.dropna(how='all').reset_index(drop=True)

# def main():
#     # Define expected headers - any one of these should be present
#     headers = [
#         'Resh ID', 'Lease ID', 'Unit', 'Floor Plan', 'Unit Designation', 'SQFT',
#         'Unit/Lease Status', 'Name', 'Phone Number', 'Email', 'Move-In', 
#         'Notice For Date', 'Move-Out', 'Lease Start', 'Lease End', 'Market + Addl.',
#         'Dep On Hand', 'Balance', 'Total Charges', 'RENT', 'PEST', 'VALET TRASH',
#         'TRASH', 'PETRENT', 'LOP15', 'EMPLCRED', 'OHIOCHOICE', 'WAIVED ADMIN FEE',
#         'WAIVED APP FEE', 'SETUPFEE'
#     ]
    
#     # Pipeline execution
#     file_path = input("Please enter the path to your rent roll Excel file: ")
#     excel_file = load_excel_data(file_path)
#     raw_data = parse_excel_data(excel_file)
#     header_index = find_header_row(raw_data, headers)
    
#     if header_index is not None:
#         # Extract raw metadata
#         metadata = extract_metadata(raw_data, header_index)
        
#         # Use GPT to analyze and structure the metadata
#         client = initialize_openai_client()
        
#         # Convert metadata to string format for GPT analysis
#         metadata_text = "\n".join([
#             " | ".join(str(v) for v in row.values()) 
#             for row in metadata.values()
#         ])
        
#         # Prepare prompts for GPT analysis
#         system_prompt = {
#             "role": "system", 
#             "content": """Analyze this rent roll metadata and extract key information into a structured format.
#             Focus on:
#             - Property name and details
#             - Report date/period
#             - Occupancy metrics
#             - Financial summaries
#             - Any other relevant property metrics
#             Format as detailed JSON with clear labels and categorized information."""
#         }
        
#         user_prompt = {
#             "role": "user",
#             "content": f"Analyze this rent roll metadata:\n\n{metadata_text}"
#         }
        
#         # Get GPT analysis
#         response = client.chat.completions.create(
#             model="gpt-4-turbo-preview",
#             messages=[system_prompt, user_prompt],
#             temperature=0.3,
#             response_format={"type": "json_object"}
#         )
        
#         # Parse GPT response and update metadata
#         analyzed_metadata = json.loads(response.choices[0].message.content)
#         metadata = {"raw_metadata": metadata, "analyzed_metadata": analyzed_metadata}
        
#         # Process rent roll data
#         rent_roll_data = process_rent_roll_data(raw_data, header_index)
        
#         print("Metadata:")
#         print(metadata.values())
        
#         return rent_roll_data
#     else:
#         raise ValueError("Could not find any of the expected headers in data")

# # Execute pipeline
# rent_roll_data = main()
# rent_roll_data


#### Meta data extractor only using the open ai 

In [56]:
# import pandas as pd
# from openai import OpenAI
# import os
# import dotenv
# import json
# from pathlib import Path
# import logging

# # # Setup logging
# # # logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# # logger = logging.getLogger(__name__)

# # def initialize_openai_client():
# #     """Initialize OpenAI client with proper error handling and configuration."""
# #     try:
# #         # Load environment variables
# #         dotenv.load_dotenv()
        
# #         # Get API key with validation
# #         api_key = os.getenv("OPENAI_API_KEY")
# #         if not api_key:
# #             raise ValueError("OpenAI API key not found in environment variables")
        
# #         # Initialize client with configuration
# #         client = OpenAI(
# #             api_key=api_key,
# #             timeout=60.0,  # Increase timeout for longer operations
# #             max_retries=3  # Automatic retries for reliability
# #         )
        
# #         logger.info("OpenAI client initialized successfully")
# #         return client
    
# #     except Exception as e:
# #         logger.error(f"Failed to initialize OpenAI client: {str(e)}")
# #         raise

# # def extract_metadata_with_gpt(file_path: str) -> dict:
# #     """
# #     Extracts metadata from an Excel rent roll file that might be located anywhere in the file,
# #     then refines it using OpenAI GPT to handle complex edge cases.
    
# #     Args:
# #         file_path (str): Path to the Excel file.
    
# #     Returns:
# #         dict: Extracted metadata as structured fields.
# #     """
# #     try:
# #         # Validate file path
# #         file_path = Path(file_path)
# #         if not file_path.exists():
# #             raise FileNotFoundError(f"File not found: {file_path}")
        
# #         # Initialize OpenAI client
# #         client = initialize_openai_client()
        
# #         # Load the Excel file
# #         xls = pd.ExcelFile(file_path)
# #         sheet_name = xls.sheet_names[0]  # Assume the first sheet is relevant
# #         df = pd.read_excel(xls, sheet_name=sheet_name, dtype=str)
        
# #         # Aggregate all non-empty rows as potential metadata
# #         all_rows = []
# #         for _ , row in df.iterrows():
# #             non_empty_values = row.dropna().to_list()
# #             if non_empty_values:
# #                 all_rows.append(" | ".join(non_empty_values))
# #         all_text = "\n".join(all_rows)
        
# #         # Prepare GPT prompt with detailed instructions
# #         system_prompt = {
# #             "role": "system",
# #             "content": (
# #                 "You are an AI that extracts structured metadata from rent roll documents. "
# #                 "The provided text contains both metadata and tabular rent roll data. "
# #                 "Ignore the tabular data and extract only metadata such as the property name, report date, "
# #                 "rent roll month/year, and any additional parameters. Format the output as valid JSON using the structure:\n\n"
# #                 '{\n'
# #                 '  "property_name": <string>,\n'
# #                 '  "report_date": <string>,\n'
# #                 '  "report_timestamp": <string>,\n'
# #                 '  "additional_info": { ... }\n'
# #                 '}'
                
# #                 "Remember: DO NOT include any totals, financial summaries, or detailed data. Only extract metadata."
# #             )
# #         }
        
# #         user_prompt = {
# #             "role": "user",
# #             "content": f"Extract metadata from the following rent roll content:\n\n{all_text}"
# #         }
        
# #         # Call OpenAI API with error handling
# #         try:
# #             response = client.chat.completions.create(
# #                 model="gpt-4-turbo-preview",
# #                 messages=[system_prompt, user_prompt],
# #                 temperature=0.3,  # Lower temperature for consistent output
# #                 response_format={"type": "json_object"}  # Request JSON response
# #             )
            
# #             extracted_metadata = response.choices[0].message.content
# #             return json.loads(extracted_metadata)
            
# #         except Exception as e:
# #             logger.error(f"OpenAI API error: {str(e)}")
# #             raise
            
# #     except Exception as e:
# #         logger.error(f"Error in metadata extraction: {str(e)}")
# #         raise

# # if __name__ == "__main__":
# #     try:
# #         file_path = input("Please enter the path to your rent roll Excel file: ")
# #         metadata_output = extract_metadata_with_gpt(file_path)
        
# #         print("\nExtracted Metadata:")
# #         print(json.dumps(metadata_output, indent=2))
        
# #     except Exception as e:
# #         logger.error(f"Program execution failed: {str(e)}")



In [58]:
import pandas as pd

def load_excel_data(file_path: str) -> pd.ExcelFile:
    """Load Excel file from given path"""
    return pd.ExcelFile(file_path)

def parse_excel_data(excel_file: pd.ExcelFile) -> pd.DataFrame:
    """Parse Excel file into DataFrame"""
    return excel_file.parse()

def find_header_row(data: pd.DataFrame, headers: list) -> int:
    """Find the row containing any of the expected headers"""
    header_row_idx = None
    for i, row in data.iterrows():
        if any(col in headers for col in row.astype(str)):  # Check if any column matches expected headers
            header_row_idx = i
            break  # Stop after finding the first match

    if header_row_idx is None:
        raise ValueError("No matching header row found in the Excel file.")
    
    print(f"Header row found at index: {header_row_idx}")
    return header_row_idx
def extract_metadata(data: pd.DataFrame, header_index: int) -> dict:
    """Extract and process metadata before header row"""
    metadata = data.iloc[:header_index].copy()
    metadata = metadata.dropna(how='all').reset_index(drop=True)
    
    metadata_dict = {}
    for idx, row in metadata.iterrows():
        row_clean = row.dropna()
        if not row_clean.empty:
            metadata_dict[f"row_{idx}"] = row_clean.to_dict()
    return metadata_dict

def process_rent_roll_data(data: pd.DataFrame, header_index: int) -> pd.DataFrame:
    """Process rent roll data after header row"""
    data = data.iloc[header_index:].reset_index(drop=True)
    data.columns = data.iloc[0]
    rent_roll_data = data.iloc[1:].reset_index(drop=True)
    return rent_roll_data.dropna(how='all').reset_index(drop=True)
def main():
    # Define expected headers - any one of these should be present
    headers = [
        'Resh ID', 'Lease ID', 'Unit', 'Floor Plan', 'Unit Designation', 'SQFT',
        'Unit/Lease Status', 'Name', 'Phone Number', 'Email', 'Move-In', 
        'Notice For Date', 'Move-Out', 'Lease Start', 'Lease End', 'Market + Addl.',
        'Dep On Hand', 'Balance', 'Total Charges', 'RENT', 'PEST', 'VALET TRASH',
        'TRASH', 'PETRENT', 'LOP15', 'EMPLCRED', 'OHIOCHOICE', 'WAIVED ADMIN FEE',
        'WAIVED APP FEE', 'SETUPFEE'
    ]
    
    # Pipeline execution
    file_path = input("Please enter the path to your rent roll Excel file: ")
    excel_file = load_excel_data(file_path)
    raw_data = parse_excel_data(excel_file)
    header_index = find_header_row(raw_data, headers)
    
    if header_index is not None:
        metadata = extract_metadata(raw_data, header_index)
        rent_roll_data = process_rent_roll_data(raw_data, header_index)
        
        print("Metadata:")
        print(metadata.values())
        
        return rent_roll_data
    else:
        raise ValueError("Could not find any of the expected headers in data")

# Execute pipeline
rent_roll_data = main()
rent_roll_data


Header row found at index: 4
Metadata:
dict_values([{'Abbie Lakes': 'RENT ROLL DETAIL MODIFIED'}, {'Abbie Lakes': '5/16/2024 3:11:55 AM'}, {'Abbie Lakes': 'As of Date: 5/15/2024'}, {'Abbie Lakes': 'Parameters: Property - ALL; Unit Designation - ALL; Subjournal - ALL; Sort By - Unit; Display - Market + Addl.'}])


,Resh ID,Lease ID,Unit,Floor Plan,Unit Designation,SQFT,Unit/Lease Status,Name,Phone Number,Email,Move-In,Notice For Date,Move-Out,Lease Start,Lease End,Market + Addl.,Dep On Hand,Balance,Total Charges,RENT,PEST,VALET TRASH,TRASH,PETRENT,LOP15,EMPLCRED,OHIOCHOICE,WAIVED ADMIN FEE,WAIVED APP FEE,SETUPFEE
0,101887,102845,3680,B2R,NaN,1032,Occupied,"Wray, Joe",614-815-9588,joepwrayjr@gmail.com,02/29/2024,NaN,NaN,02/29/2024,02/20/2025,1414.00,300.00,0.00,1338.00,1296.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,91810,92712,3682,B2R,NaN,1032,Occupied,"Wright, Patty",(614) 634-8523,pwright3611@gmail.com,07/17/2023,NaN,NaN,07/17/2023,09/16/2024,1459.00,300.00,0.00,1372.00,1331.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,101838,102758,3684,B2R,NaN,1032,Occupied,"Barry, Abdoulaye",646-403-7710,bademba.barrie89@live.com,08/02/2023,NaN,NaN,08/02/2023,10/30/2024,1364.00,300.00,0.00,1393.00,1337.00,3.00,30.00,8.00,0.00,15.00,0.00,0.00,0.00,0.00,0.00
3,101936,102913,3686,B2R,NaN,1032,Occupied,"Ivory, Rashida",347-255-8785,ivoryrashida@gmail.com,03/11/2024,NaN,NaN,03/11/2024,03/10/2025,1409.00,300.00,0.00,1380.00,1303.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
4,101939,102918,3688,B1R,NaN,990,Occupied,"Cooper, Tyreke",(614) 556-7982,tyreke68@gmail.com,03/20/2024,NaN,NaN,03/20/2024,04/17/2025,1343.00,300.00,0.00,1339.00,1297.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,101855,102886,3690,B1R,NaN,990,Occupied,"Lawrence, Lisa",614-204-1704,gmalisa17@gmail.com,09/07/2023,NaN,NaN,04/05/2024,11/11/2024,1363.00,300.00,-1508.57,1364.00,1287.00,3.00,31.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
6,91679,102829,3692,A1R,NaN,806,Occupied-NTVL,"Bouillon, Douglas",419-937-5272,dbouillon07@yahoo.com,10/04/2022,06/05/2024,NaN,01/06/2024,06/05/2024,1189.00,300.00,-224.49,1272.00,1181.00,3.00,30.00,8.00,35.00,15.00,0.00,0.00,0.00,0.00,0.00
7,101948,102936,3694,A1R,NaN,806,Occupied,"Sanker, Matthew",380-250-8029,ggggoal1957@gmail.com,04/06/2024,NaN,NaN,04/06/2024,05/07/2025,1169.00,300.00,0.00,1169.00,1127.00,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,30,92577,3696,B1R,NaN,990,Occupied,"Smith, William",(614) 216-1689,willwsmith1966@gmail.com,08/01/2011,NaN,NaN,04/09/2023,06/08/2024,1293.00,199.00,0.00,1220.00,1179.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,50784,92698,3698,B1R,NaN,990,Occupied,"Cruz, Marlene",(614) 984-4116,cruzmarlene044@gmail.com,02/03/2018,NaN,NaN,07/24/2023,07/16/2024,1313.00,100.00,0.00,1264.00,1223.00,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
